In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [2]:
import json

with open('config.json') as f:
    conf = json.load(f)
    host = conf['host']
    database = conf['database']
    user = conf['user']
    passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)
conn = psycopg2.connect(conn_str)

In [4]:
query = '''
SELECT      DISTINCT inc.incident_id AS INCIDENT_ID,
            vic.victim_id AS VICTIM_ID,
            date_part('year',inc.incident_date),
            vic.victim_type_id AS VICTIM_TYPE_ID,
            ty.victim_type_name AS VICTIM_TYPE,
            vic.age_num,
            vic.age_range_low_num AS AGE_RANGE_LOW,
            vic.age_range_high_num AS AGE_RANGE_HIGH,
            vic.age_id,
            age.age_code,
            age.age_name,
            vic.sex_code AS VICTIM_SEX,
            oft.crime_against AS CRIME_AGAINST,
            oft.offense_name AS OFFENSE,
            oft.offense_category_name AS OFFENSE_CATEGORY,
            oft.offense_group AS OFFENSE_GROUP,
            off.location_id AS LOCATION_ID,
            loc.location_name AS LOCATION_NAME,
            ori.fips AS FIPS,
            ori.countyname AS COUNTY,
            ori.name AS ORI_NAME,
            ags.population AS POPULATION,
            ags.population_group_desc AS POPULATION_DESCRIPTION,
            ags.total_officers AS OFFICERS,
            ags.total_civilians AS CIVILIANS
            
FROM        nibrs_victim as vic
JOIN        nibrs_victim_type as ty
ON          vic.victim_type_id = ty.victim_type_id
JOIN        nibrs_age as age
ON          age.age_id = vic.age_id
JOIN        nibrs_offense as off
ON          off.incident_id = vic.incident_id
JOIN        nibrs_offense_type as oft
ON          oft.offense_type_id = off.offense_type_id
JOIN        nibrs_location_type as loc
ON          off.location_id = loc.location_id
JOIN        nibrs_incident as inc
ON          inc.incident_id = vic.incident_id
JOIN        cde_agencies as ags
ON          ags.agency_id = inc.agency_id
JOIN        ori_to_fips as ori
ON          ori.ori9 = ags.ori

WHERE       vic.victim_type_id = 4
AND         inc.incident_date BETWEEN '2014-01-01' AND '2016-12-31';
'''

In [5]:
df = pd.read_sql(query, con=conn)
df.head(3).T

,0,1,2
incident_id,73180647,73180648,73180649
victim_id,79599709,79599710,79599711
date_part,2014,2014,2014
victim_type_id,4,4,4
victim_type,Individual,Individual,Individual
age_num,35,60,38
age_range_low,NaN,NaN,NaN
age_range_high,NaN,NaN,NaN
age_id,5,5,5
age_code,AG,AG,AG


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478607 entries, 0 to 478606
Data columns (total 25 columns):
incident_id               478607 non-null int64
victim_id                 478607 non-null int64
date_part                 478607 non-null float64
victim_type_id            478607 non-null int64
victim_type               478607 non-null object
age_num                   469263 non-null float64
age_range_low             184635 non-null float64
age_range_high            184635 non-null float64
age_id                    478607 non-null int64
age_code                  478607 non-null object
age_name                  478607 non-null object
victim_sex                478607 non-null object
crime_against             478607 non-null object
offense                   478607 non-null object
offense_category          478607 non-null object
offense_group             478607 non-null object
location_id               478607 non-null int64
location_name             478607 non-null object
fips    